# Content Based Learning

In [1]:
#importing necessary libraries
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from geopy.distance import great_circle

In [5]:
# loading data
import pandas as pd
df = pd.read_excel('dataset_tourism.xlsx')
# show data
print(df.head())


   Place_Id                          Place_Name  \
0        29           Wisata Kuliner Pecenongan   
1        69                     Freedom Library   
2        95  Desa Wisata Sungai Code Jogja Kota   
3       124        Kauman Pakualaman Yogyakarta   
4       278       Masjid Daarut Tauhiid Bandung   

                                         Description        City  \
0  Pecenongan merupakan salah satu surga kuliner ...     Jakarta   
1  Freedom Library adalah perpustakaan buku yang ...     Jakarta   
2  Kampung Code berada di Kelurahan Kotabaru, Kec...  Yogyakarta   
3  Pakualaman adalah sebuah kecamatan di Kota Yog...  Yogyakarta   
4  pada bulan Juli 1993 berdirilah sebuah masjid ...     Bandung   

             Category  Price       Lat        Long  Rating  
0  Pusat Perbelanjaan      0 -6.166789  106.826526     5.0  
1              Budaya      0 -6.202248  106.845197     5.0  
2       Taman Hiburan      0 -7.822909  110.375689     5.0  
3              Budaya      0 -7.801035  

In [6]:
# struktur data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 576 entries, 0 to 575
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Place_Id     576 non-null    int64  
 1   Place_Name   576 non-null    object 
 2   Description  576 non-null    object 
 3   City         576 non-null    object 
 4   Category     576 non-null    object 
 5   Price        576 non-null    int64  
 6   Lat          576 non-null    float64
 7   Long         576 non-null    float64
 8   Rating       576 non-null    float64
dtypes: float64(3), int64(2), object(4)
memory usage: 40.6+ KB


In [7]:
# statdes
df.describe()

,Place_Id,Price,Lat,Long,Rating
count,576.000000,576.000000,576.000000,576.000000,576.000000
mean,289.531250,25434.027778,-6.896178,111.221720,4.375521
std,168.117037,58426.214978,1.393292,6.622427,0.273416
min,1.000000,0.000000,-10.726300,103.931398,3.400000
25%,144.750000,2000.000000,-7.784302,107.592150,4.200000
50%,288.500000,10000.000000,-7.003838,110.362640,4.400000
75%,432.250000,25000.000000,-6.610755,110.557625,4.600000
max,592.000000,900000.000000,1.078880,140.718700,5.000000


In [9]:
# check missing value
missing_values = df.isnull().sum()

# menampilkan kolom-kolom dengan nilai yang hilang (jika ada)
print("Column Missing Value:")
print(missing_values[missing_values > 0])


Column Missing Value:
Series([], dtype: int64)


In [10]:
df.isnull().sum()

Place_Id       0
Place_Name     0
Description    0
City           0
Category       0
Price          0
Lat            0
Long           0
Rating         0
dtype: int64

In [41]:
#Check Data Duplicate
duplicate = df[df.duplicated(subset=['Place_Name','City'])]

if duplicate.empty:
    print("No duplicate")
else:
    print("Duplicate")
    print(duplicate)

Duplicate
     Place_Id                Place_Name  \
61        488            Pantai Natsepa   
268       585            Taman Suropati   
469       545        Keraton Yogyakarta   
473       569   Taman Pintar Yogyakarta   
521       514  Bukit Bintang Yogyakarta   
525       579           Museum Nasional   
545       519           Candi Prambanan   
546       534             Gunung Merapi   
552       583             Dunia Fantasi   
564       529          Gembira Loka Zoo   
570       586      Grand Indonesia Mall   
575       512           Ancol Dreamland   

                                           Description        City  \
61   Bersantai di Pantai Natsepa, pantai dengan air...       Ambon   
268  Bersantai di Taman Suropati yang hijau dan dam...     Jakarta   
469  Kunjungi Keraton Yogyakarta, istana tradisiona...  Yogyakarta   
473  Tempat edukasi dan hiburan, Taman Pintar Yogya...  Yogyakarta   
521  Menyaksikan Yogyakarta dari ketinggian di Buki...  Yogyakarta   
525  Menel

In [42]:
df = df.drop_duplicates(subset=['Place_Name','City'], keep='first')
df

,Place_Id,Place_Name,Description,City,Category,Price,Lat,Long,Rating
0,29,Wisata Kuliner Pecenongan,Pecenongan merupakan salah satu surga kuliner ...,Jakarta,Pusat Perbelanjaan,0,-6.166789,106.826526,5.0
1,69,Freedom Library,Freedom Library adalah perpustakaan buku yang ...,Jakarta,Budaya,0,-6.202248,106.845197,5.0
2,95,Desa Wisata Sungai Code Jogja Kota,"Kampung Code berada di Kelurahan Kotabaru, Kec...",Yogyakarta,Taman Hiburan,0,-7.822909,110.375689,5.0
3,124,Kauman Pakualaman Yogyakarta,Pakualaman adalah sebuah kecamatan di Kota Yog...,Yogyakarta,Budaya,0,-7.801035,110.375192,5.0
4,278,Masjid Daarut Tauhiid Bandung,pada bulan Juli 1993 berdirilah sebuah masjid ...,Bandung,Tempat Ibadah,0,-6.863527,107.589919,4.9
...,...,...,...,...,...,...,...,...,...
569,580,Kota Tua Jakarta,"Jelajahi Kota Tua Jakarta, area bersejarah den...",Jakarta,Budaya,14000,-6.135000,106.813300,3.7
571,592,Summarecon Mall Serpong,Berbelanja dan bersantap di Summarecon Mall Se...,Tangerang,Pusat Perbelanjaan,13000,-6.242000,106.627300,3.7
572,515,Bukit Cinta,Menikmati pemandangan matahari terbenam yang r...,Magelang,Taman Hiburan,8000,-7.510600,110.227200,3.5
573,573,Tebing Keraton,Trekking menuju Tebing Keraton dengan pemandan...,Bandung,Taman Hiburan,12000,-6.752800,107.612700,3.5


Flow:
1. **Dataset Understanding:**
   - Analyze the columns in the dataset, such as `Place_Id`, `Place_Name`, `Description`, `City`, `Category`, `Price`, `Lat`, `Long`, and `Rating`.
   -Ensure the data is clean, and perform preprocessing such as changing data types, handling null values, or performing normalization.


2. **Feature Selection:**
   - Selecting the features to be used in the model. In this case, `Description`, `City`, `Category`, `Price`, and `Rating` will most likely be the relevant features for content-based recommendation.


3. **Text Preprocessing:**
   - Preprocessing the text fields, such as cleaning the text from special characters, converting letters to lowercase, and tokenizing.
   - Using tools such as TF-IDF or Word Embeddings to convert text into numerical representations.


4. **Location Feature Conversion (Lat, Long):**
   - Since we want to incorporate location information into the model, it is necessary to convert `Lat` and `Long` into more informative location features, such as distance from city center or location cluster.


5. **Feature Normalization:**
   - Normalize the values of numeric features, such as `Price` and `Rating`, to ensure that all features have a similar scale.


6. **Creation of Similarity Matrix:**
   - Calculate the similarity matrix between tourist attractions based on the features that have been selected. In this case, a method such as cosine similarity can be used.


7. **Selection of Recommended Places:**
   - Determine the tourist attractions that you want to recommend. This can be user input or can be selected based on some specific criteria.


8. **Ranking and Recommendation:**
   - Rank the tourist attractions based on the similarity value with the previously selected places, and recommend the places with the highest similarity value.


9. **Model Evaluation:**
   - Evaluate the model by dividing the dataset into training and test data. Some evaluation metrics that can be used include precision, recall, or Mean Squared Error (MSE).


10. **System Implementation:**
   - Implement the recommendation model into a full system that includes 360 view, photo, video, location, and travel companion features.

In [43]:
# Fitur Selection
selected_features = ['Description', 'City', 'Category', 'Price', 'Rating']
df_selected = df[selected_features]
print(df_selected.head())


                                         Description        City  \
0  Pecenongan merupakan salah satu surga kuliner ...     Jakarta   
1  Freedom Library adalah perpustakaan buku yang ...     Jakarta   
2  Kampung Code berada di Kelurahan Kotabaru, Kec...  Yogyakarta   
3  Pakualaman adalah sebuah kecamatan di Kota Yog...  Yogyakarta   
4  pada bulan Juli 1993 berdirilah sebuah masjid ...     Bandung   

             Category  Price  Rating  
0  Pusat Perbelanjaan      0     5.0  
1              Budaya      0     5.0  
2       Taman Hiburan      0     5.0  
3              Budaya      0     5.0  
4       Tempat Ibadah      0     4.9  


In [44]:
# Text Processing
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# Text preprocessing
df_selected.loc[:, 'Description'] = df_selected['Description'].str.lower()  # Using .loc to modify values

# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(df_selected['Description'])

# Displaying some examples of TF-IDF Vectorization results
print("Example of TF-IDF Vectorization results:")
print(tfidf_matrix[:2])  # Displaying the first two rows as examples


Example of TF-IDF Vectorization results:
  (0, 3168)	0.09485353660125692
  (0, 2839)	0.09748967971058581
  (0, 372)	0.06387496246313734
  (0, 4484)	0.12062827423805564
  (0, 5694)	0.0802547088580505
  (0, 2716)	0.05630595074126456
  (0, 3126)	0.13992687478422125
  (0, 4779)	0.12532804704101486
  (0, 4828)	0.08848700074522949
  (0, 5847)	0.07388817300202308
  (0, 4330)	0.07736502598069676
  (0, 2544)	0.07299179702713221
  (0, 2267)	0.09748967971058581
  (0, 5940)	0.07255752068510443
  (0, 1368)	0.09251094049303762
  (0, 6175)	0.10218945251354501
  (0, 3417)	0.11354161160626605
  (0, 3706)	0.22145843859561687
  (0, 2740)	0.13992687478422125
  (0, 2643)	0.09251094049303762
  (0, 3619)	0.07530859808449956
  (0, 4486)	0.11354161160626605
  (0, 2530)	0.08848700074522949
  (0, 788)	0.09613039155460203
  (0, 2228)	0.07130596502175376
  :	:
  (1, 6048)	0.12751393931820798
  (1, 5627)	0.12253528239166155
  (1, 1381)	0.061915097126999684
  (1, 5108)	0.16483022674261924
  (1, 1766)	0.1648302267426

Interpretation:
The displayed TF-IDF Vectorization result is a numerical representation of two documents or tourist attractions in the dataset. Each row represents one document, and each column represents a particular word or token in the entire dataset. The value in each cell represents the TF-IDF weight of that word in that document.

The example shows that certain words have a high TF-IDF weight in the first document, while other words have a lower weight. The same applies to the second document. This process helps to quantify the importance of a word in a document relative to the entire dataset, thus enabling similarity comparison between documents based on this vector representation. At a later stage, this TF-IDF matrix can be used to measure the similarity between tourist attractions, which is useful for building content-based recommendation models.

In [45]:
# Feature selection
selected_features = ['Description', 'City', 'Category', 'Price', 'Rating', 'Lat', 'Long']
df_selected = df[selected_features]


In [46]:
# Cleaning and Numeric Value Conversion for 'Lat'
df_selected['Lat'] = df_selected['Lat'].replace(',', '.', regex=True).astype(float)

# Cleaning and Numeric Value Conversion for 'Long'
df_selected['Long'] = df_selected['Long'].replace(',', '.', regex=True).astype(float)


C:\Users\DELL\AppData\Local\Temp\ipykernel_10116\1705724195.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['Lat'] = df_selected['Lat'].replace(',', '.', regex=True).astype(float)
C:\Users\DELL\AppData\Local\Temp\ipykernel_10116\1705724195.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['Long'] = df_selected['Long'].replace(',', '.', regex=True).astype(float)


In [47]:

# Text preprocessing
df_selected['Description'] = df_selected['Description'].str.lower()

# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(df_selected['Description'])

C:\Users\DELL\AppData\Local\Temp\ipykernel_10116\1978108491.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['Description'] = df_selected['Description'].str.lower()


In [48]:
# Features Conversion
# Using groupby and agg to calculate the average 'Lat' and 'Long' per city
city_center = df_selected.groupby('City').agg({'Lat': 'mean', 'Long': 'mean'}).mean().values
latitude_city_center, longitude_city_center = city_center

# City center coordinates (replace with actual coordinates)
city_center = (latitude_city_center, longitude_city_center)

# Function to calculate distance from the city center
def calculate_distance(row):
    place_coordinates = (float(row['Lat']), float(row['Long']))
    return great_circle(city_center, place_coordinates).km

# Adding the 'Distance_from_city_center' column to the DataFrame
df_selected['Distance_from_city_center'] = df_selected.apply(calculate_distance, axis=1)

# Displaying the first few rows of the updated DataFrame
print(df_selected.head())


                                         Description        City  \
0  pecenongan merupakan salah satu surga kuliner ...     Jakarta   
1  freedom library adalah perpustakaan buku yang ...     Jakarta   
2  kampung code berada di kelurahan kotabaru, kec...  Yogyakarta   
3  pakualaman adalah sebuah kecamatan di kota yog...  Yogyakarta   
4  pada bulan juli 1993 berdirilah sebuah masjid ...     Bandung   

             Category  Price  Rating       Lat        Long  \
0  Pusat Perbelanjaan      0     5.0 -6.166789  106.826526   
1              Budaya      0     5.0 -6.202248  106.845197   
2       Taman Hiburan      0     5.0 -7.822909  110.375689   
3              Budaya      0     5.0 -7.801035  110.375192   
4       Tempat Ibadah      0     4.9 -6.863527  107.589919   

   Distance_from_city_center  
0                1333.532460  
1                1331.390451  
2                 954.969529  
3                 954.613897  
4                1249.879414  


C:\Users\DELL\AppData\Local\Temp\ipykernel_10116\496577508.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['Distance_from_city_center'] = df_selected.apply(calculate_distance, axis=1)


In [49]:
# Features Normalization
from sklearn.preprocessing import MinMaxScaler

# Initialize the scaler
scaler = MinMaxScaler()

# Normalize the 'Price' and 'Distance_from_city_center' features
df_selected[['Price', 'Distance_from_city_center']] = scaler.fit_transform(df_selected[['Price', 'Distance_from_city_center']])

# Displaying the first few rows of the updated DataFrame
print(df_selected.head())


                                         Description        City  \
0  pecenongan merupakan salah satu surga kuliner ...     Jakarta   
1  freedom library adalah perpustakaan buku yang ...     Jakarta   
2  kampung code berada di kelurahan kotabaru, kec...  Yogyakarta   
3  pakualaman adalah sebuah kecamatan di kota yog...  Yogyakarta   
4  pada bulan juli 1993 berdirilah sebuah masjid ...     Bandung   

             Category  Price  Rating       Lat        Long  \
0  Pusat Perbelanjaan    0.0     5.0 -6.166789  106.826526   
1              Budaya    0.0     5.0 -6.202248  106.845197   
2       Taman Hiburan    0.0     5.0 -7.822909  110.375689   
3              Budaya    0.0     5.0 -7.801035  110.375192   
4       Tempat Ibadah    0.0     4.9 -6.863527  107.589919   

   Distance_from_city_center  
0                   0.489640  
1                   0.488665  
2                   0.317350  
3                   0.317188  
4                   0.451568  


C:\Users\DELL\AppData\Local\Temp\ipykernel_10116\2397919692.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected[['Price', 'Distance_from_city_center']] = scaler.fit_transform(df_selected[['Price', 'Distance_from_city_center']])


Normalisasi membantu dalam menangani skala yang berbeda-beda antar fitur, sehingga memastikan bahwa semua fitur berkontribusi sebanding dalam perhitungan kesamaan.

In [50]:
# Combining TF-IDF Vectorization results with other features and building a content-based recommendation model.
# In this case, we can use the TF-IDF matrix as the numerical representation of the description of tourist attractions,
# while the 'Price' and 'Distance_from_city_center' features have been normalized.

from scipy.sparse import hstack

# Combining the TF-IDF matrix with other features
X = hstack([tfidf_matrix, df_selected[['Price', 'Distance_from_city_center']].values])

# Convert the matrix to CSR (Compressed Sparse Row) format for indexing
X_array = X.tocsr()

# Convert the matrix to a dense NumPy array format
X_array = X.todense()

# Displaying the first few rows of the combined matrix
print(X_array[:2])


[[0.         0.         0.         ... 0.         0.         0.48964004]
 [0.         0.         0.         ... 0.         0.         0.48866518]]


Interpretation:

The interpretation of the displayed results is the first two rows of the combined matrix after normalizing and combining the features from the TF-IDF Vectorization matrix, 'Price', and 'Distance_from_city_center'.


Each element in the matrix shows the numerical representation value of the combination of these features for each tourist attraction. The two rows represent two different tourist attractions in the dataset.


An example of interpreting one element of the matrix:


- The element in the first row and the last column (0.51263763): This value represents how similar the first tourist spot is to itself based on the similarity of its features.


- Element in the second row and last column (0.51165502): This value represents how similar the second tourist spot is to itself.


Keep in mind that these values are the result of calculating the cosine similarity between the vectors representing the tourist attractions in the matrix. The higher the cosine similarity value, the more similar the two tourist attractions are.


With this matrix, we can proceed to build a content-based recommendation system. If you have any further questions or want to proceed with the next steps, let me know!

Next, we can use the calculated cosine similarity matrix to build a content-based recommendation system. In the content-based filtering approach, we can recommend tourist attractions that are similar to places that have been liked by users or that are currently being viewed by users.

The following are the steps to perform the recommendation:

- Identifying Places to Recommend:

- Select one or more tourist attractions that we want to use as a reference for recommendation. In this example, we will use one tourist spot as an example.
Get the Index of the Selected Venue:

- Finds the index of the selected place in the dataset.
Calculate Cosine Similarity:

- Uses the cosine similarity matrix to calculate the similarity between the selected place and all other places in the dataset.
Sort and Select Recommended Places:

- Sort the similarity results and select some places with the highest similarity as recommendations.

# Recommendations System

In [51]:
# Selecting one tourist attraction as an example (replace with the appropriate index)
index_selected_place = 0

# Calculating cosine similarity between tourist attractions
cosine_sim = cosine_similarity(X, X)

# Calculating cosine similarity between the selected place and other places
similarity_scores = cosine_sim[index_selected_place]

# Getting the indices of some places with the highest similarity
top_recommendations_indices = similarity_scores.argsort()[-5:-1][::-1]

# Displaying the recommended places
print("Recommended places:")
for i, index in enumerate(top_recommendations_indices, 1):
    print(f"{i}. {df['Place_Name'].iloc[index]}")


Recommended places:
1. Kawasan Kuliner BSM
2. Pelabuhan Marina
3. Kampung Koki
4. Kampung Nafri


In [52]:
# Choosing one tourist attraction as an example (replace with the appropriate index)
index_selected_place = 1

# Calculating cosine similarity between tourist attractions
cosine_sim = cosine_similarity(X, X)

# Calculating cosine similarity between the selected place and other places
similarity_scores = cosine_sim[index_selected_place]

# Getting the indices of some places with the highest similarity
top_recommendations_indices = similarity_scores.argsort()[-5:-1][::-1]

# Displaying the recommended places
print("Recommended places:")
for i, index in enumerate(top_recommendations_indices, 1):
    print(f"{i}. {df['Place_Name'].iloc[index]}")


Recommended places:
1. Perpustakaan Nasional
2. Pantai Tablanusu
3. Taman Ekspresi Dan Perpustakaan
4. Gua Bintang


In [53]:
# Choosing one tourist attraction as an example (replace with the appropriate index)
index_selected_place = 0

# Calculating cosine similarity between the selected place and other places
similarity_scores = cosine_sim[index_selected_place]

# Specifying the desired number of recommendations
number_recomendation = 5

# Getting the indices of places with the highest similarity
top_recommendations_indices = similarity_scores.argsort()[-number_recomendation:][::-1]

# Displaying the recommended places
print(f"{jumlah_rekomendasi} Recommended places:")
for i, index in enumerate(top_recommendations_indices, 1):
    print(f"{i}. {df['Place_Name'].iloc[index]}")


5 Recommended places:
1. Wisata Kuliner Pecenongan
2. Kawasan Kuliner BSM
3. Pelabuhan Marina
4. Kampung Koki
5. Kampung Nafri
